#All install and imports

In [84]:
#install
!pip install pymupdf
!pip install sentence-transformers



In [85]:
#imports
from sentence_transformers import SentenceTransformer, util
import fitz  # PyMuPDF
import re
import unicodedata
import re



#PDF

##Functions

###Extract pdf resume

In [86]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    full_text = ""
    for page in doc:
        full_text += page.get_text() + "\n"
    return full_text


###Initial cleaning

In [87]:
def clean_resume_text(text):
    # Normalize Unicode (e.g. accents, special characters)
    text = unicodedata.normalize("NFKD", text)

    # Remove zero-width spaces and non-printable characters
    text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)

    # Remove multiple spaces and fix inconsistent newlines
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{2,}', '\n', text)

    # Strip each line
    lines = [line.strip() for line in text.splitlines() if line.strip()]
    return '\n'.join(lines)

###Sectioning

In [121]:
def split_resume_into_sections(text):
    sections = {
        "contact": "",
        "summary": "",
        "skills": "",
        "experience": "",
        "education": "",
        "certifications": "",
        "projects": "",
        "others": ""
    }

    patterns = {
        "summary": r"(summary|professional summary|profile)",
        "skills": r"(skills|technical skills)",
        "experience": r"(experience|work experience|employment history)",
        "education": r"(education|academic background)",
        "certifications": r"(certifications|licenses)",
        "projects": r"(projects|project highlights)",
        "others": r"(interests|languages|volunteer|publications|additional information)"
    }

    lines = text.splitlines()
    current_section = "contact"  # Assume resume starts with contact

    for line in lines:
        clean_line = line.strip().lower()

        # Check if this line is a section header
        for section, pattern in patterns.items():
            if re.search(rf"^{pattern}[:\s]*$", clean_line):
                current_section = section
                break
        else:
            sections[current_section] += line + '\n'

    return sections


###Second cleaning-sectioned

In [89]:
# def clean_resume_section(text):
#     # Normalize unicode
#     text = unicodedata.normalize("NFKD", text)
#     # Remove invisible characters
#     text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)
#     # Remove common bullet characters and excess punctuation
#     text = re.sub(r'[•●◦▪️]', '', text)
#     text = re.sub(r'[-–—]', ' ', text)
#     # Normalize whitespace
#     text = re.sub(r'\s+', ' ', text).strip()
#     # Ensure proper sentence endings
#     # text = re.sub(r'(?<!\w)([A-Z][a-z]+)', r'\n\1', text)  # crude line breaks before new capitalized words
#     return text.strip()
# def prepare_sections_for_semantic_scoring(section_dict):
#     cleaned_sections = {}
#     for section, content in section_dict.items():
#         cleaned_text = clean_resume_section(content)
#         cleaned_sections[section] = cleaned_text
#     return cleaned_sections

In [140]:
import re
import unicodedata

def clean_resume_section(text):
    # Unicode normalization
    text = unicodedata.normalize("NFKD", text)

    # Remove zero-width spaces and similar non-printables
    text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)

    # Standardize bullet points (replace all types with `-`)
    text = re.sub(r'[•●◦▪️]', '-', text)

    # Normalize dashes (– — − → -)
    text = re.sub(r'[–—−]', '-', text)

    # Fix spaces around separators
    text = re.sub(r'\s*[-]\s*', ' - ', text)

    # Remove leading/trailing hyphens if orphaned
    text = re.sub(r'^[-\s]+|[-\s]+$', '', text)

    # Remove duplicate dashes (e.g., "--")
    text = re.sub(r'-{2,}', '-', text)

    # Ensure single spaces
    text = re.sub(r'\s+', ' ', text)

    # Preserve line breaks where they indicate structure
    text = text.replace('. ', '.\n')  # Break long lines into per-sentence structure

    # Final strip
    return text.strip()
def prepare_sections_for_semantic_scoring(section_dict):
    cleaned_sections = {}
    for section, content in section_dict.items():
        cleaned_text = clean_resume_section(content)
        cleaned_sections[section] = cleaned_text
    return cleaned_sections


###Convert to natural languange

In [138]:
# def others_list_to_sentence(skill_text):
#     # Remove bullets and split by common delimiters
#     skills = re.split(r'[,\n;]', skill_text)
#     skills = [s.strip() for s in skills if s.strip()]
#     if not skills:
#         return ""
#     return "Others things i have participate in thats relevant fot this role are " + ", ".join(skills[:-1]) + f", and {skills[-1]}."
# def certifications_list_to_sentence(skill_text):
#     # Remove bullets and split by common delimiters
#     skills = re.split(r'[,\n;]', skill_text)
#     skills = [s.strip() for s in skills if s.strip()]
#     if not skills:
#         return ""
#     return "I hold certifications including " + ", ".join(skills[:-1]) + f", and {skills[-1]}."
# def skill_list_to_sentence(skill_text):
#     # Remove bullets and split by common delimiters
#     skills = re.split(r'[,\n;]', skill_text)
#     skills = [s.strip() for s in skills if s.strip()]
#     if not skills:
#         return ""

#     return "This candidate is proficient in " + ", ".join(skills[:-1]) + f", and {skills[-1]}."
# def bullet_points_to_sentences(text):
#     bullets = re.split(r'\n|•|- ', text)
#     sentences = []
#     for bullet in bullets:
#         b = bullet.strip()
#         if b and not b.endswith('.'):
#             b += '.'
#         if b:
#             sentences.append(b)
#     return " ".join(sentences)
# #========================================================
# def naturalize_resume_sections(sections):
#     natural_sections = {}
#     for section, content in sections.items():
#         if section == 'skills':
#             natural_sections[section] = skill_list_to_sentence(content)
#         elif section == 'others':
#             natural_sections[section] = others_list_to_sentence(content)
#         elif section == 'certifications':
#             natural_sections[section] = certifications_list_to_sentence(content)
#         elif section in ['experience', 'projects']:
#             natural_sections[section] = bullet_points_to_sentences(content)
#         else:
#             natural_sections[section] = content  # already in sentence form
#     return natural_sections

In [145]:
import re

def bullet_points_to_sentences(text):
    # Split by common bullet formats (dash, bullet, newline)
    bullets = re.split(r'(?:\n|^)[\-\•●◦▪️] ?', text)
    sentences = []
    for bullet in bullets:
        b = bullet.strip()
        if not b:
            continue
        # Ensure the sentence ends with a period
        if not b.endswith('.'):
            b += '.'
        sentences.append(b)
    return ' '.join(sentences)

def skill_list_to_sentence(skill_text):
    skills = re.split(r'[,\n;]', skill_text)
    skills = [s.strip() for s in skills if s.strip()]
    if not skills:
        return ""
    if len(skills) == 1:
        return f"This candidate is proficient in {skills[0]}."
    return "This candidate is proficient in " + ", ".join(skills[:-1]) + f", and {skills[-1]}."

def certifications_list_to_sentence(cert_text):
    certs = re.split(r'[,\n;]', cert_text)
    certs = [c.strip().rstrip('.') for c in certs if c.strip()]
    if not certs:
        return ""
    if len(certs) == 1:
        return f"I hold the certification: {certs[0]}."
    return "I hold certifications including " + ", ".join(certs[:-1]) + f", and {certs[-1]}."

def others_list_to_sentence(others_text):
    items = re.split(r'[,\n;]', others_text)
    items = [i.strip().rstrip('.') for i in items if i.strip()]
    if not items:
        return ""
    if len(items) == 1:
        return f"Other relevant participation includes {items[0]}."
    return "Other relevant participation includes " + ", ".join(items[:-1]) + f", and {items[-1]}."

# ==========================================

def naturalize_resume_sections(sections):
    natural_sections = {}
    for section, content in sections.items():
        if section == 'skills':
            natural_sections[section] = skill_list_to_sentence(content)
        elif section == 'others':
            natural_sections[section] = others_list_to_sentence(content)
        elif section == 'certifications':
            natural_sections[section] = certifications_list_to_sentence(content)
        elif section in ['experience', 'projects']:
            natural_sections[section] = bullet_points_to_sentences(content)
        else:
            # Clean up rogue spacing/punctuation in regular paragraphs
            content = content.strip()
            if content and not content.endswith('.'):
                content += '.'
            natural_sections[section] = content
    return natural_sections


###Final cleaning

jadi manfaatin delimiter untuk bedain header, bullet points dll. setelah sudah dpt bentuk natural languange baru dilakukan final cleaning

In [146]:
# def clean_text(text):
#     # text = unidecode.unidecode(text)  # Remove accents
#     text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
#     text = re.sub(r'[^a-zA-Z0-9.,;!?()\[\] ]+', '', text)  # Remove special characters
#     # text = text.lower()  # Lowercase
#     return text
# #cleaning dict values
# def clean_dict_values(data_dict):
#     return {key: clean_text(value) for key, value in data_dict.items()}
# # Assume `sections` is the output from your previous dictionary splitting function

In [156]:
def clean_text(text):
    text = re.sub(r'\s*[-–—]\s*', '-', text)  # Restore hyphens like front-end
    text = re.sub(r'\|\s*', ', ', text)       # Convert pipes to commas
    text = re.sub(r'\s{2,}', ' ', text)       # Collapse excessive spacing
    text = re.sub(r'\s*\.\s*', '. ', text)    # Ensure space after periods
    text = re.sub(r'\s*,\s*', ', ', text)     # Normalize commas
    text = re.sub(r'\.\s*\.', '.', text)      # Remove double periods
    text = text.strip()
    return text
# #cleaning dict values
def clean_dict_values(data_dict):
    return {key: clean_text(value) for key, value in data_dict.items()}


##Master functions

In [157]:
def pdf_processing(file_path): #nanti di app.py panggil dgn pdf_file.name
  resume_raw=extract_text_from_pdf(file_path)
  #initial cleaning
  cleaned=clean_resume_text(resume_raw)
  #sectioning
  sections = split_resume_into_sections(cleaned)
  #second cleaning for the sectioned text
  cleaned_sections = prepare_sections_for_semantic_scoring(sections)
  #natural langunage format
  naturalized=naturalize_resume_sections(cleaned_sections)
  # final cleaning ( final result)
  final = clean_dict_values(naturalized)
  return final

##Test

In [158]:
pdf_path='MOCK-Alex_Johnson_Resume.pdf'
pdf_resume_preprocessed=pdf_processing(pdf_path)

In [160]:
pdf_resume_preprocessed

{'contact': 'Alex Johnson (123) 456-7890, alexjohnson. dev@email. com, github. com/alexjohnson-dev, alexjohnson. dev.',
 'summary': 'Junior Web Developer with a strong foundation in front-end and back-end technologies. Skilled in building responsive, user-centric websites and applications. Passionate about learning new technologies and delivering high-quality code.',
 'skills': 'This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.',
 'experience': 'Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap. Built RESTful APIs with Node. js and Express. js. Created mobile-first responsive designs with UX teams. Improved website performance by 20%. Freelance Web Dev

#Txt

Expecting a copy paste from user

##Functions

### Initial Cleaning

In [95]:
# import unicodedata
# import re

# def clean_jd_text(text):
#     # Normalize Unicode (e.g. accents, special characters)
#     text = unicodedata.normalize("NFKD", text)

#     # Remove zero-width spaces and non-printable characters
#     text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)

#     # Remove multiple spaces and fix inconsistent newlines
#     text = re.sub(r'[ \t]+', ' ', text)
#     text = re.sub(r'\n{2,}', '\n', text)

#     # Strip each line
#     lines = [line.strip() for line in text.splitlines() if line.strip()]
#     return '\n'.join(lines)


In [96]:
import unicodedata
import re

def clean_jd_text(text):
    # 1. Unicode normalize (NFKD breaks accents into base + diacritic)
    text = unicodedata.normalize("NFKD", text)

    # 2. Remove zero-width/non-printable characters
    text = re.sub(r'[\u200B-\u200D\uFEFF]', '', text)

    # 3. Convert curly quotes, en/em dashes, and ellipses to standard equivalents
    text = text.replace("’", "'").replace("‘", "'")\
               .replace("“", '"').replace("”", '"')\
               .replace("–", "-").replace("—", "-")\
               .replace("…", "...")

    # 4. Remove multiple spaces/tabs, normalize newlines
    text = re.sub(r'[ \t]+', ' ', text)
    text = re.sub(r'\n{2,}', '\n', text)

    # 5. Strip leading/trailing spaces per line, remove empty lines
    lines = [line.strip() for line in text.splitlines() if line.strip()]

    # 6. Optional: ensure a period ends any standalone sentence-like lines (good before sectioning)
    lines = [line if re.search(r'[.!?]$', line) else line + '.' for line in lines]

    return '\n'.join(lines)


###Sectioning

In [97]:
# def split_into_sections(text):
#     sections = {
#         "job info": "",
#         "summary": "",
#         "responsibilities": "",
#         "requirements": "",
#         "education":"",
#         "preferred_qualifications": ""
#     }

#     patterns = {
#         "summary": r"(about the role)",
#         "responsibilities": r"(responsibilities)",
#         "requirements": r"(requirements)",
#         "education": r"(education)",
#         "preferred_qualifications": r"(interests|languages|volunteer|publications|additional information|certifications|nice to have|certifications / nice to have)"
#     }

#     lines = text.splitlines()
#     current_section = "job info"  # Assume resume starts with basic info of the job

#     for line in lines:
#         clean_line = line.strip().lower()

#         # Check if this line is a section header
#         for section, pattern in patterns.items():
#             if re.search(rf"^{pattern}[:\s]*$", clean_line):
#                 current_section = section
#                 break
#         else:
#             sections[current_section] += line + '\n'

#     return sections


In [98]:
# import re

def split_into_sections(text):
    sections = {
        "job info": "",
        "summary": "",
        "responsibilities": "",
        "requirements": "",
        "education": "",
        "preferred_qualifications": ""
    }

    # Slightly expanded, but still safe and not overfitted
    patterns = {
        "summary": r"\b(about the role|summary|job overview)\b",
        "responsibilities": r"\b(responsibilities|what you’ll do|your tasks)\b",
        "requirements": r"\b(requirements|qualifications|must haves|you have)\b",
        "education": r"\b(education|academic background)\b",
        "preferred_qualifications": r"\b(preferred qualifications|nice to have|certifications|additional information|interests|languages|volunteer|publications)\b"
    }

    lines = text.splitlines()
    current_section = "job info"

    for line in lines:
        clean_line = line.strip().lower()

        # Detect section header
        matched = False
        for section, pattern in patterns.items():
            if re.search(pattern, clean_line):
                current_section = section
                matched = True
                break

        # If no header match, keep appending to current section
        if not matched:
            sections[current_section] += line + '\n'

    # Final cleanup
    for key in sections:
        sections[key] = sections[key].strip()

    return sections


###Final cleaning

In [99]:
# def clean_text(text):
#     # text = unidecode.unidecode(text)  # Remove accents
#     text = re.sub(r'\s+', ' ', text)  # Normalize whitespace
#     text = re.sub(r'[^a-zA-Z0-9.,;!?()\[\] ]+', '', text)  # Remove special characters

#     # sentences = re.sub(r'-\s*', '', text).strip()
#     # paragraph= re.sub(r'\n+','. ',sentences)
#     # if not paragraph.endswith('.'):
#     #   paragraph+='.'
#     # text = text.lower()  # Lowercase
#     return text
# #cleaning dict values
# def clean_dict_values(data_dict):
#     return {key: clean_text(value) for key, value in data_dict.items()}
# # Assume `sections` is the output from your previous dictionary splitting function



In [100]:
import re

def clean_text(text):
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text.strip())

    # Add period to end of bullet points and sentences if missing
    # (Handles '-', '*', or other list formats)
    lines = re.split(r'(?<=\n)|(?<=\.) ', text)
    processed_lines = []

    for line in lines:
        line = line.strip()

        # Add period if it's a bullet or sentence and doesn't end properly
        if line:
            if line.startswith(('-', '*')):
                if not line.endswith('.'):
                    line += '.'
            elif not re.search(r'[.!?]$', line):
                line += '.'
            processed_lines.append(line)

    # Rejoin the lines as a single string
    text = ' '.join(processed_lines)

    # Remove unwanted special characters except those useful for sentence context
    text = re.sub(r'[^a-zA-Z0-9.,;!?()\-\'\" ]+', '', text)

    return text

def clean_dict_values(data_dict):
    return {key: clean_text(value) for key, value in data_dict.items()}


##Master functions

In [101]:
def txt_processor(text):
  jd_raw=text
  jd_cleaned=clean_jd_text(jd_raw)
  jd_sectioned=split_into_sections(jd_cleaned)
  final = clean_dict_values(jd_sectioned)
  return final

##Test

In [102]:
jd_text="""Job Title: Junior Web Developer
Location: Remote (US-based preferred)
Employment Type: Full-Time

About the Role
We are seeking a passionate Junior Web Developer to join our growing team. This role is ideal for someone who enjoys building responsive, user-friendly web applications and working in a collaborative, agile environment. You will support front-end and back-end development tasks, contribute to code reviews, and help ensure our applications perform efficiently across devices.

Responsibilities
- Develop and maintain responsive web pages using HTML5, CSS3, and JavaScript
- Work with React.js on the front end and Node.js + Express.js on the back end
- Design and consume RESTful APIs
- Collaborate with UI/UX designers to implement mobile-first designs
- Use version control systems like Git and platforms such as GitHub
- Write clean, maintainable, and well-documented code
- Participate in Agile workflows including daily standups and sprint planning

Requirements
- Proficiency in HTML5, CSS3, and JavaScript (ES6+)
- Familiarity with React.js, Node.js, Express.js
- Basic understanding of MongoDB and MySQL
- Experience with Git, GitHub, VS Code, and Postman
- Comfortable working with APIs and debugging front-end/back-end issues
- Strong communication and problem-solving skills
- Willingness to learn and grow in a fast-paced development environment

Education
Bachelor’s degree in Computer Science or related field (or equivalent practical experience)

Certifications / Nice to Have
- Completion of freeCodeCamp certifications or similar online programs
- Internship or freelance experience building real-world web applications
- Exposure to tools like Figma, Firebase, or Vercel
"""

###Run

In [103]:
txt_jd_preprocessed=txt_processor(jd_text)
txt_jd_preprocessed

{'job info': 'Job Title Junior Web Developer. Location Remote (US-based preferred). Employment Type Full-Time.',
 'summary': 'We are seeking a passionate Junior Web Developer to join our growing team. This role is ideal for someone who enjoys building responsive, user-friendly web applications and working in a collaborative, agile environment. You will support front-end and back-end development tasks, contribute to code reviews, and help ensure our applications perform efficiently across devices.',
 'responsibilities': '- Develop and maintain responsive web pages using HTML5, CSS3, and JavaScript. - Work with React.js on the front end and Node.js  Express.js on the back end. - Design and consume RESTful APIs. - Collaborate with UIUX designers to implement mobile-first designs. - Use version control systems like Git and platforms such as GitHub. - Write clean, maintainable, and well-documented code. - Participate in Agile workflows including daily standups and sprint planning.',
 'requi

In [104]:
#jd sectioned

In [105]:
#jd cleaned

#Semantic Scoring

##Function

In [106]:

# model = SentenceTransformer('all-MiniLM-L6-v2')  # Efficient and accurate
model = SentenceTransformer('all-mpnet-base-v2')


def compute_semantic_similarity(resume_dict, job_dict):
    scores = {}

    # Define section pairs to compare
    section_pairs = [
        ('summary', 'summary'),
        ('skills', 'requirements'),
        ('experience', 'responsibilities'),
        ('certifications', 'preferred_qualifications'),
        ('projects', 'responsibilities'),
        ('others', 'preferred_qualifications')
    ]

    for resume_key, job_key in section_pairs:
        resume_text = resume_dict.get(resume_key, "").strip()
        job_text = job_dict.get(job_key, "").strip()

        if resume_text and job_text:
            emb1 = model.encode(resume_text, convert_to_tensor=True)
            emb2 = model.encode(job_text, convert_to_tensor=True)
            similarity = util.cos_sim(emb1, emb2).item()
            scores[f"{resume_key} vs {job_key}"] = round(similarity, 4)
        else:
            scores[f"{resume_key} vs {job_key}"] = None  # missing data

    return scores


##get the files

In [161]:
job_description=txt_jd_preprocessed.copy()
resume=pdf_resume_preprocessed.copy()

In [162]:
job_description

{'job info': 'Job Title Junior Web Developer. Location Remote (US-based preferred). Employment Type Full-Time.',
 'summary': 'We are seeking a passionate Junior Web Developer to join our growing team. This role is ideal for someone who enjoys building responsive, user-friendly web applications and working in a collaborative, agile environment. You will support front-end and back-end development tasks, contribute to code reviews, and help ensure our applications perform efficiently across devices.',
 'responsibilities': '- Develop and maintain responsive web pages using HTML5, CSS3, and JavaScript. - Work with React.js on the front end and Node.js  Express.js on the back end. - Design and consume RESTful APIs. - Collaborate with UIUX designers to implement mobile-first designs. - Use version control systems like Git and platforms such as GitHub. - Write clean, maintainable, and well-documented code. - Participate in Agile workflows including daily standups and sprint planning.',
 'requi

In [163]:
resume

{'contact': 'Alex Johnson (123) 456-7890, alexjohnson. dev@email. com, github. com/alexjohnson-dev, alexjohnson. dev.',
 'summary': 'Junior Web Developer with a strong foundation in front-end and back-end technologies. Skilled in building responsive, user-centric websites and applications. Passionate about learning new technologies and delivering high-quality code.',
 'skills': 'This candidate is proficient in Languages: HTML5, CSS3, JavaScript (ES6+), Python Frameworks/Libraries: React. js, Node. js, Express. js, Bootstrap Databases: MongoDB, MySQL Tools: Git, GitHub, Visual Studio Code, Figma, Postman Other: RESTful APIs, Responsive Design, and Agile Methodology.',
 'experience': 'Web Development Intern, BrightWave Solutions, Remote, Jan 2024-Apr 2024-Developed and maintained front-end components using React. js and Bootstrap. Built RESTful APIs with Node. js and Express. js. Created mobile-first responsive designs with UX teams. Improved website performance by 20%. Freelance Web Dev

##compute

In [164]:
similarity_scores = compute_semantic_similarity(resume, job_description)
for k, v in similarity_scores.items():
    print(f"{k}: {v}")


summary vs summary: 0.8699
skills vs requirements: 0.7566
experience vs responsibilities: 0.5541
certifications vs preferred_qualifications: 0.5657
projects vs responsibilities: 0.6302
others vs preferred_qualifications: 0.3649
